In [4]:
%run ../functions_scripts/database_functions.ipynb
%run ../functions_scripts/dataframe_cleaning_functions.ipynb

In [5]:

from pathlib import Path
from typing import List, Optional
from sqlalchemy import create_engine
from sqlalchemy.engine import Engine
from sqlalchemy import text
from psycopg2.extras import execute_values
from sqlalchemy_utils import database_exists, create_database
import pandas as pd

import os
from dotenv import load_dotenv

load_dotenv()

RAW_DIR = Path(os.getenv("RAW_DIR"))
CLEAN_DIR = Path(os.getenv("CLEAN_DIR"))
ADMIN_DB_URL = os.getenv("ADMIN_DB_URL")
DB_URL = os.getenv("DB_URL")



# 🧹 Nettoyage de la table `caracteristiques`


In [6]:
# Etablir connection avec la base de données
engine = get_engine(DB_URL) 

In [7]:
#Afficher le nombre de lignes dans la table bronze.caracteristiques_raw

query = "SELECT COUNT(*) FROM bronze.caracteristiques_raw"
count= get_single_value(engine, query)
print(f"Nombre de lignes dans bronze.caracteristiques_raw : {count}")
print('\n')

#Récupérer les données dans Dataframe df_caracteristiques
df_caracteristiques = get_df_from_table(engine, "bronze", "caracteristiques_raw")
df_caracteristiques.head()

Nombre de lignes dans bronze.caracteristiques_raw : 475911




,identifiant_de_l_accident,date_et_heure,jour,mois,annee,heure_minute,date,year_georef,lumiere,code_postal,...,localisation,intersection,conditions_atmospheriques,collision,adresse,gps,latitude,longitude,coordonnees,numero
0,201700033619,2017-05-19T17:40:00+02:00,19,05,2017,17:40,None,2017,Plein jour,66100,...,Hors agglomération,6,Normale,Deux véhicules – par l’arrière,COPENHAGUE (ROND POINT D,Métropole,4269110,0285200,"42.668322, 2.897617",None
1,201700048262,2017-12-25T17:55:00+01:00,25,12,2017,17:55,None,2017,Crépuscule ou aube,93200,...,Hors agglomération,1,Normale,Trois véhicules et plus – en chaîne,AUTOROUTE A1,Métropole,None,None,None,1
2,201700048288,2017-01-01T17:25:00+01:00,01,01,2017,17:25,None,2017,Nuit sans éclairage public,92230,...,Hors agglomération,1,Temps couvert,Trois véhicules et plus – en chaîne,A86,None,None,None,"48.925197, 2.293085",86
3,201700048486,2017-05-04T00:04:00+02:00,04,05,2017,00:04,None,2017,Nuit avec éclairage public non allumé,78330,...,Hors agglomération,1,Autre,Autre collision,A12 Y,Métropole,4880700,0205200,"48.812878, 2.048119",12
4,201700048639,2017-07-11T08:10:00+02:00,11,07,2017,08:10,None,2017,Plein jour,78140,...,Hors agglomération,1,Normale,Deux véhicules – par le coté,N118,Métropole,4878430,0222310,"48.783326, 2.187486",118


In [8]:
# supprimer les colonnes avec plusieurs missing values (dépasse la moitié de valeurs)
df_caracteristiques.drop(columns=['date'], inplace=True)
# creer liste de colonnes supprimées
cols_supprimees= ['date']

In [9]:
# Supprimer les doublons
print(f"Nombre de lignes avant suppression des doublons : {df_caracteristiques.shape[0]}")
df_caracteristiques = df_caracteristiques.drop_duplicates()
print(f"Nombre de lignes après suppression des doublons : {df_caracteristiques.shape[0]}")
#remplacer les valeurs nulles
remplacer_valeurs_nulles(df_caracteristiques)

Nombre de lignes avant suppression des doublons : 475911
Nombre de lignes après suppression des doublons : 475911


,identifiant_de_l_accident,date_et_heure,jour,mois,annee,heure_minute,year_georef,lumiere,code_postal,code_insee,...,localisation,intersection,conditions_atmospheriques,collision,adresse,gps,latitude,longitude,coordonnees,numero
0,201700033619,2017-05-19T17:40:00+02:00,19,05,2017,17:40,2017,Plein jour,66100,66136,...,Hors agglomération,6,Normale,Deux véhicules – par l’arrière,COPENHAGUE (ROND POINT D,Métropole,4269110,0285200,"42.668322, 2.897617",Inconnu
1,201700048262,2017-12-25T17:55:00+01:00,25,12,2017,17:55,2017,Crépuscule ou aube,93200,93066,...,Hors agglomération,1,Normale,Trois véhicules et plus – en chaîne,AUTOROUTE A1,Métropole,Inconnu,Inconnu,Inconnu,1
2,201700048288,2017-01-01T17:25:00+01:00,01,01,2017,17:25,2017,Nuit sans éclairage public,92230,92036,...,Hors agglomération,1,Temps couvert,Trois véhicules et plus – en chaîne,A86,Inconnu,Inconnu,Inconnu,"48.925197, 2.293085",86
3,201700048486,2017-05-04T00:04:00+02:00,04,05,2017,00:04,2017,Nuit avec éclairage public non allumé,78330,78242,...,Hors agglomération,1,Autre,Autre collision,A12 Y,Métropole,4880700,0205200,"48.812878, 2.048119",12
4,201700048639,2017-07-11T08:10:00+02:00,11,07,2017,08:10,2017,Plein jour,78140,78640,...,Hors agglomération,1,Normale,Deux véhicules – par le coté,N118,Métropole,4878430,0222310,"48.783326, 2.187486",118
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475906,201600004642,2016-01-01T05:00:00+01:00,01,01,2016,05:00,2016,Nuit sans éclairage public,01290,01046,...,En agglomération,1,Brouillard - fumée,Deux véhicules - frontale,Les Allouets,Métropole,Inconnu,Inconnu,"46.21663, 4.951376",Inconnu
475907,201600004875,2016-07-27T16:15:00+02:00,27,07,2016,16:15,2016,Plein jour,38530,38314,...,En agglomération,9,Normale,Autre collision,parking av de la gare,Métropole,Inconnu,Inconnu,"45.433123, 6.015207",Inconnu
475908,201600000781,2016-01-14T08:00:00+01:00,14,01,2016,08:00,2016,Nuit avec éclairage public allumé,27400,27003,...,En agglomération,2,Pluie forte,Deux véhicules - frontale,RD 164,Métropole,Inconnu,Inconnu,Inconnu,164
475909,201400047990,2014-03-08T17:15:00+01:00,08,03,2014,17:15,2015,Plein jour,92210,92064,...,Hors agglomération,1,Normale,Trois véhicules et plus - collisions multiples,A13,Inconnu,Inconnu,Inconnu,"48.845569, 2.215131",13


In [10]:
#Nettoyage sémantique 
#Traiter valeurs spéciales (-1, 00, NULL chaîne).
df_caracteristiques =  traiter_valeurs_speciales(df_caracteristiques)
df_caracteristiques.head()


,identifiant_de_l_accident,date_et_heure,jour,mois,annee,heure_minute,year_georef,lumiere,code_postal,code_insee,...,localisation,intersection,conditions_atmospheriques,collision,adresse,gps,latitude,longitude,coordonnees,numero
0,201700033619,2017-05-19T17:40:00+02:00,19,05,2017,17:40,2017,Plein jour,66100,66136,...,Hors agglomération,6,Normale,Deux véhicules – par l’arrière,COPENHAGUE (ROND POINT D,Métropole,4269110,0285200,"42.668322, 2.897617",None
1,201700048262,2017-12-25T17:55:00+01:00,25,12,2017,17:55,2017,Crépuscule ou aube,93200,93066,...,Hors agglomération,1,Normale,Trois véhicules et plus – en chaîne,AUTOROUTE A1,Métropole,None,None,None,1
2,201700048288,2017-01-01T17:25:00+01:00,01,01,2017,17:25,2017,Nuit sans éclairage public,92230,92036,...,Hors agglomération,1,Temps couvert,Trois véhicules et plus – en chaîne,A86,None,None,None,"48.925197, 2.293085",86
3,201700048486,2017-05-04T00:04:00+02:00,04,05,2017,00:04,2017,Nuit avec éclairage public non allumé,78330,78242,...,Hors agglomération,1,Autre,Autre collision,A12 Y,Métropole,4880700,0205200,"48.812878, 2.048119",12
4,201700048639,2017-07-11T08:10:00+02:00,11,07,2017,08:10,2017,Plein jour,78140,78640,...,Hors agglomération,1,Normale,Deux véhicules – par le coté,N118,Métropole,4878430,0222310,"48.783326, 2.187486",118


In [11]:
#changer type de colonne date_et_heure de texte vers datetime avec fuseau temps UTC+00 mondial
df_caracteristiques['date_et_heure'] =pd.to_datetime(

    df_caracteristiques['date_et_heure'],
    errors='raise',
    utc=True
    ).to_frame()
df_caracteristiques

,identifiant_de_l_accident,date_et_heure,jour,mois,annee,heure_minute,year_georef,lumiere,code_postal,code_insee,...,localisation,intersection,conditions_atmospheriques,collision,adresse,gps,latitude,longitude,coordonnees,numero
0,201700033619,2017-05-19 15:40:00+00:00,19,05,2017,17:40,2017,Plein jour,66100,66136,...,Hors agglomération,6,Normale,Deux véhicules – par l’arrière,COPENHAGUE (ROND POINT D,Métropole,4269110,0285200,"42.668322, 2.897617",None
1,201700048262,2017-12-25 16:55:00+00:00,25,12,2017,17:55,2017,Crépuscule ou aube,93200,93066,...,Hors agglomération,1,Normale,Trois véhicules et plus – en chaîne,AUTOROUTE A1,Métropole,None,None,None,1
2,201700048288,2017-01-01 16:25:00+00:00,01,01,2017,17:25,2017,Nuit sans éclairage public,92230,92036,...,Hors agglomération,1,Temps couvert,Trois véhicules et plus – en chaîne,A86,None,None,None,"48.925197, 2.293085",86
3,201700048486,2017-05-03 22:04:00+00:00,04,05,2017,00:04,2017,Nuit avec éclairage public non allumé,78330,78242,...,Hors agglomération,1,Autre,Autre collision,A12 Y,Métropole,4880700,0205200,"48.812878, 2.048119",12
4,201700048639,2017-07-11 06:10:00+00:00,11,07,2017,08:10,2017,Plein jour,78140,78640,...,Hors agglomération,1,Normale,Deux véhicules – par le coté,N118,Métropole,4878430,0222310,"48.783326, 2.187486",118
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475906,201600004642,2016-01-01 04:00:00+00:00,01,01,2016,05:00,2016,Nuit sans éclairage public,01290,01046,...,En agglomération,1,Brouillard - fumée,Deux véhicules - frontale,Les Allouets,Métropole,None,None,"46.21663, 4.951376",None
475907,201600004875,2016-07-27 14:15:00+00:00,27,07,2016,16:15,2016,Plein jour,38530,38314,...,En agglomération,9,Normale,Autre collision,parking av de la gare,Métropole,None,None,"45.433123, 6.015207",None
475908,201600000781,2016-01-14 07:00:00+00:00,14,01,2016,08:00,2016,Nuit avec éclairage public allumé,27400,27003,...,En agglomération,2,Pluie forte,Deux véhicules - frontale,RD 164,Métropole,None,None,None,164
475909,201400047990,2014-03-08 16:15:00+00:00,08,03,2014,17:15,2015,Plein jour,92210,92064,...,Hors agglomération,1,Normale,Trois véhicules et plus - collisions multiples,A13,None,None,None,"48.845569, 2.215131",13


In [12]:
#changer le jour , mois en decimale
df_caracteristiques['jour'] = df_caracteristiques['jour'].astype(float) 
df_caracteristiques['mois'] = df_caracteristiques['mois'].astype(float)
#changer l'annee en int
df_caracteristiques['annee'] = df_caracteristiques['annee'].astype(int)

In [13]:
# standardiser la colonne lumiere
#Standardiser casse/accents si nécessaire.
df_caracteristiques = standardiser_colonne_caisse(df_caracteristiques, 'lumiere', lower=True, strip=True, remove_accents=True)

In [14]:
#Harmonisation de la colonne 'intersection'
print(df_caracteristiques.groupby('intersection').size().reset_index(name='counts').sort_values(by='counts', ascending=False))
mapping_intersection = {
    '0': 'non renseigné',
    '1': 'hors intersection',
    '2': 'intersection en x',
    '3': 'intersection en t',
    '4': 'intersection en y',
    '5': 'intersection à plus de 4 branches',
    '6': 'giratoire',
    '7': 'place',
    '8': 'passage à niveau',
    '9': 'autre intersection'
}
harmonise_colonne_df(df_caracteristiques,'intersection',mapping_intersection)
print('\napres harmonisation:')
print(df_caracteristiques.groupby('intersection').size().reset_index(name='counts').sort_values(by='counts', ascending=False))

  intersection  counts
1            1  328937
2            2   56882
3            3   44672
6            6   15386
9            9   10060
4            4    8361
7            7    6100
5            5    4833
8            8     590
0            0      90

apres harmonisation:
                        intersection  counts
2                  hors intersection  328937
4                  intersection en x   56882
3                  intersection en t   44672
1                          giratoire   15386
0                 autre intersection   10060
5                  intersection en y    8361
9                              place    6100
6  intersection à plus de 4 branches    4833
8                   passage à niveau     590
7                      non renseigné      90


In [15]:
# harmoniser colonne collision et remplacer -1 par Inconnu
print(df_caracteristiques.groupby('collision').size().reset_index(name='counts').sort_values(by='counts', ascending=False))

df_caracteristiques = remplacer_valeur(
    df_caracteristiques,
    'collision',
    '-1',
    'Inconnu'
)
print(df_caracteristiques.groupby('collision').size().reset_index(name='counts').sort_values(by='counts', ascending=False))

                                        collision  counts
0                                 Autre collision  161298
2                    Deux véhicules – par le coté  132605
3                  Deux véhicules – par l’arrière   58352
4                                  Sans collision   46028
1                       Deux véhicules - frontale   45192
6             Trois véhicules et plus – en chaîne   16916
5  Trois véhicules et plus - collisions multiples   15508
                                        collision  counts
0                                 Autre collision  161298
2                    Deux véhicules – par le coté  132605
3                  Deux véhicules – par l’arrière   58352
4                                  Sans collision   46028
1                       Deux véhicules - frontale   45192
6             Trois véhicules et plus – en chaîne   16916
5  Trois véhicules et plus - collisions multiples   15508


In [16]:
#convertir la colonne coordonnees en tuples (latitude, longitude) de type float
def parse_coord(x):
    if x is None or pd.isna(x) or ',' not in str(x):
        return (None, None)
    parts = str(x).split(',')
    lat = float(parts[0].strip())
    lon = float(parts[1].strip())
    return (lat, lon)

df_caracteristiques['coordonnees_tuple'] = df_caracteristiques['coordonnees'].apply(parse_coord)
# Vérification
df_caracteristiques[['coordonnees', 'coordonnees_tuple']].head()

,coordonnees,coordonnees_tuple
0,"42.668322, 2.897617","(42.668322, 2.897617)"
1,None,"(None, None)"
2,"48.925197, 2.293085","(48.925197, 2.293085)"
3,"48.812878, 2.048119","(48.812878, 2.048119)"
4,"48.783326, 2.187486","(48.783326, 2.187486)"


In [17]:

#supprimer les anciens colonne latitude et longitude
df_caracteristiques.drop(columns=['latitude', 'longitude','coordonnees'], inplace=True)

# Créer latitude et longitude à partir de coordonnees_tuple
df_caracteristiques['latitude'] = df_caracteristiques['coordonnees_tuple'].apply(lambda x: x[0] if x else None)
df_caracteristiques['longitude'] = df_caracteristiques['coordonnees_tuple'].apply(lambda x: x[1] if x else None)
#supprimer la colonne coordonnees_tuple
df_caracteristiques.drop(columns=['coordonnees_tuple'], inplace=True)


cols_supprimees.append('coordonnees')
df_caracteristiques.head()

,identifiant_de_l_accident,date_et_heure,jour,mois,annee,heure_minute,year_georef,lumiere,code_postal,code_insee,...,nom_officiel_epci,localisation,intersection,conditions_atmospheriques,collision,adresse,gps,numero,latitude,longitude
0,201700033619,2017-05-19 15:40:00+00:00,19.0,5.0,2017,17:40,2017,plein jour,66100,66136,...,CU Perpignan Méditerranée Métropole,Hors agglomération,giratoire,Normale,Deux véhicules – par l’arrière,COPENHAGUE (ROND POINT D,Métropole,None,42.668322,2.897617
1,201700048262,2017-12-25 16:55:00+00:00,25.0,12.0,2017,17:55,2017,crépuscule ou aube,93200,93066,...,Métropole du Grand Paris,Hors agglomération,hors intersection,Normale,Trois véhicules et plus – en chaîne,AUTOROUTE A1,Métropole,1,NaN,NaN
2,201700048288,2017-01-01 16:25:00+00:00,1.0,1.0,2017,17:25,2017,nuit sans éclairage public,92230,92036,...,Métropole du Grand Paris,Hors agglomération,hors intersection,Temps couvert,Trois véhicules et plus – en chaîne,A86,None,86,48.925197,2.293085
3,201700048486,2017-05-03 22:04:00+00:00,4.0,5.0,2017,00:04,2017,nuit avec éclairage public non allumé,78330,78242,...,CA Versailles Grand Parc (C.A.V.G.P.),Hors agglomération,hors intersection,Autre,Autre collision,A12 Y,Métropole,12,48.812878,2.048119
4,201700048639,2017-07-11 06:10:00+00:00,11.0,7.0,2017,08:10,2017,plein jour,78140,78640,...,CA Versailles Grand Parc (C.A.V.G.P.),Hors agglomération,hors intersection,Normale,Deux véhicules – par le coté,N118,Métropole,118,48.783326,2.187486


In [18]:
#supprimer les colonnes supprimées de la table caracteristiques_clean dans silver
drop_columns(engine, "silver", "caracteristiques_clean", cols_supprimees)

#inserer les données nettoyées dans la table silver.caracteristiques_clean
truncate_table_if_exists(engine, "silver", "caracteristiques_clean")
insert_df_to_table(engine, df_caracteristiques, "silver", "caracteristiques_clean")

Colonne supprimée : date
Colonne supprimée : coordonnees
Table caracteristiques_clean.silver vidée si elle existait.


475911

In [19]:
#Fermeture des connexions actives à la base de données
admin_engine = get_engine(ADMIN_DB_URL)
show_open_connections(admin_engine, DB_NAME)
close_all_connections(admin_engine, DB_NAME)
show_open_connections(admin_engine, DB_NAME)

INFO:__main__:Toutes les connexions à 'db_accident' ont été fermées (sauf la session actuelle).
INFO:__main__:Engine SQLAlchemy libéré — plus aucune connexion active.


(195, 'postgres', 'db_accident', 'idle', 'COMMIT')
